# 通过时间反向传播
:label:`sec_bptt`

到目前为止，我们已经多次提到
*爆炸梯度*，
*消失梯度*，
以及*需要分离*RNN的梯度.
例如, 在 :numref:`sec_rnn_scratch`中
我们调用了序列上的 `detach` 函数。
为了能够快速构建模型并了解其工作原理，所有这些都没有得到充分的解释。
在本节中，
我们将更深入地研究一下
详细介绍了序列模型的反向传播，以及数学原理。

当我们第一次实现rnn(:numref:`sec_rnn_scratch`)时，我们遇到了梯度爆炸的一些影响。
特别是，
如果你解决了这些练习，
您会看到，梯度剪裁对于确保适当的融合是至关重要的。
为了更好地理解这一问题，本文将回顾如何计算序列模型的梯度。
笔记
它的工作原理在概念上没有什么新的东西。 毕竟，我们仍然只是应用链式规则来计算梯度。 尽管如此，情况确实如此
再次查看反向传播（：numref:`sec_backprop`）时值得。


我们已经描述了向前和向后传播
和计算图
在MLPs中 :numref:`sec_backprop`。
RNN中的前向传播是相对稳定
易懂的
*通过时间*的反向传播实际上是一个特定的过程
反向传播的应用
在RNNs中 :cite:`Werbos.1990`。
它要求我们将RNN的计算图每一步展开，以获得模型变量和参数之间的依赖关系。
然后，
根据链式法则，
我们将反向传播应用于计算和分析
存储梯度。
由于序列可能相当长，因此依赖关系可能相当长。
例如，对于1000个字符的序列， 
第一个标记可能会对最终位置的标记产生重大影响。
这在计算上并不可行
（这需要太长的时间和太多的内存）并且需要1000多个矩阵积，我们才能达到非常难以捉摸的梯度。
这是一个充满计算和统计不确定性的过程。
下面我们将阐明发生了什么
以及如何在实践中解决这一问题。

## RNN中的梯度分析
:label:`subsec_bptt_analysis`

我们从RNN工作原理的简化模型开始。
此模型忽略有关隐藏状态的细节以及如何更新隐藏状态的详细信息。
这里的数学符号
没有明确区分
标量， 向量, 和以前一样的矩阵。
这些细节对分析不重要
只会使符号变得混乱

在这个简化模型中，
我们表示 $h_t$ 作为隐藏状态，
$x_t$ 作为输入， 和 $o_t$ 作为输出
偶而 $t$.
回想一下我们在
:numref:`subsec_rnn_w_hidden_states`
输入和隐藏状态
可以连接到
乘以隐藏层中的一个权重变量。
因此，我们使用 $w_h$ 和 $w_o$ 来
分别指示隐藏层和输出层的权重。
因此，每一步骤的隐藏状态和输出可以解释为

$$\begin{aligned}h_t &= f(x_t, h_{t-1}, w_h),\\o_t &= g(h_t, w_o),\end{aligned}$$
:eqlabel:`eq_bptt_ht_ot`

其中，$f$和$g$分别是隐藏层和输出层的转换。
因此，我们有一个价值观 $\{\ldots, (x_{t-1}, h_{t-1}, o_{t-1}), (x_{t}, h_{t}, o_t), \ldots\}$ 通过循环计算相互依赖。
正向传播相当简单。
我们所需要的就是循环通过三次 $（x_t，h_t，o_t）$ 每一步骤。
输出 $o_t$ 和所需标签 $y_t$ 之间的差异由目标函数进行评估
跨越所有的 $T$ 时间
像
$$L(x_1, \ldots, x_T, y_1, \ldots, y_T, w_h, w_o) = \frac{1}{T}\sum_{t=1}^T l(y_t, o_t).$$



对于反向传播，问题有点棘手，特别是当我们计算关于目标函数 $L$ 的参数 $w_h$ 的梯度时。具体来说，根据链式规则，

$$\begin{aligned}\frac{\partial L}{\partial w_h}  & = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)}{\partial w_h}  \\& = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)}{\partial o_t} \frac{\partial g(h_t, w_h)}{\partial h_t}  \frac{\partial h_t}{\partial w_h}.\end{aligned}$$
:eqlabel:`eq_bptt_partial_L_wh`

第一个和第二个因素
中的乘积:eqref:`eq_bptt_partial_L_wh`
很容易计算。
第三个因素 $\partial h_t/\partial w_h$ 这就是事情变得棘手的地方，因为我们需要反复计算参数 $w_h$ 对 $h_t$ 的影响。
根据递归计算
:eqref:`eq_bptt_ht_ot`,
$h_t$ 同时依赖于 $h_{t-1}$ 和 $w_h$,
其中，$h_{t-1}$ 的计算
还取决于 $w_h$。
因此
使用链式法则可以得出

$$\frac{\partial h_t}{\partial w_h}= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} +\frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h}.$$
:eqlabel:`eq_bptt_partial_ht_wh_recur`


为了推导上面的梯度，假设我们有三个序列 $\{a_{t}\},\{b_{t}\},\{c_{t}\}$ 满足
$a_{0}=0$ and $a_{t}=b_{t}+c_{t}a_{t-1}$ for $t=1, 2,\ldots$.
那么 $t\geq 1$，很容易展示

$$a_{t}=b_{t}+\sum_{i=1}^{t-1}\left(\prod_{j=i+1}^{t}c_{j}\right)b_{i}.$$
:eqlabel:`eq_bptt_at`

将 $a_t$, $b_t$, 和 $c_t$ 替换为
根据

$$\begin{aligned}a_t &= \frac{\partial h_t}{\partial w_h},\\
b_t &= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h}, \\
c_t &= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}},\end{aligned}$$

网格中的梯度计算 :eqref:`eq_bptt_partial_ht_wh_recur` 满足
$a_{t}=b_{t}+c_{t}a_{t-1}$.
因此，
per :eqref:`eq_bptt_at`,
我们可以在 :eqref:`eq_bptt_partial_ht_wh_recur` 中删除递归计算
具有

$$\frac{\partial h_t}{\partial w_h}=\frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h}+\sum_{i=1}^{t-1}\left(\prod_{j=i+1}^{t} \frac{\partial f(x_{j},h_{j-1},w_h)}{\partial h_{j-1}} \right) \frac{\partial f(x_{i},h_{i-1},w_h)}{\partial w_h}.$$
:eqlabel:`eq_bptt_partial_ht_wh_gen`

虽然我们可以使用链规则递归地计算 $\partial h_t/\partial w_h$ 但只要 $t$ 很长，这个链就会变得很长。让我们讨论一些处理这个问题的策略。

### 完整计算 ### 

明显地，
我们只需要计算一个完整的和
:eqref:`eq_bptt_partial_ht_wh_gen`.
然而
这是非常缓慢的，梯度可能会爆炸，
因为初始条件的细微变化可能会对结果产生很大影响。
也就是说，我们可以看到类似于蝴蝶效应的情况，即初始条件的微小变化会导致结果的不相称变化。
就我们想要估计的模型而言，这实际上是非常不可取的。
毕竟，我们正在寻找能够很好地推广的稳健估计。因此，这种策略几乎从未在实践中使用过。

### 截断时间步长 ###

或者，
我们可以把总和截断成整数
:eqref:`eq_bptt_partial_ht_wh_gen`
在 $\tau$ 这一步之后
这就是我们到目前为止一直在讨论的问题，
such as when we detached the gradients in :numref:`sec_rnn_scratch`. 
This leads to an *approximation* of the true gradient, simply by terminating the sum at 
$\partial h_{t-\tau}/\partial w_h$. 
In practice this works quite well. It is what is commonly referred to as truncated backpropgation through time :cite:`Jaeger.2002`.
One of the consequences of this is that the model focuses primarily on short-term influence rather than long-term consequences. This is actually *desirable*, since it biases the estimate towards simpler and more stable models.

### Randomized Truncation ### 

Last, we can replace $\partial h_t/\partial w_h$
by a random variable which is correct in expectation but  truncates the sequence.
This is achieved by using a sequence of $\xi_t$
with predefined $0 \leq \pi_t \leq 1$,
where $P(\xi_t = 0) = 1-\pi_t$ and  $P(\xi_t = \pi_t^{-1}) = \pi_t$, thus $E[\xi_t] = 1$.
We use this to replace the gradient
$\partial h_t/\partial w_h$
in :eqref:`eq_bptt_partial_ht_wh_recur`
with

$$z_t= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} +\xi_t \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h}.$$


It follows from the definition of $\xi_t$ that $E[z_t] = \partial h_t/\partial w_h$.
Whenever $\xi_t = 0$ the recurrent computation
terminates at that time step $t$.
This leads to a weighted sum of sequences of varying lengths where long sequences are rare but appropriately overweighted. 
This idea was proposed by Tallec and Ollivier
:cite:`Tallec.Ollivier.2017`.

### Comparing Strategies

![Comparing strategies for computing gradients in RNNs. From top to bottom: randomized truncation, regular truncation, and full computation.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/truncated-bptt.svg)
:label:`fig_truncated_bptt`


:numref:`fig_truncated_bptt` illustrates the three strategies when analyzing the first few characters of *The Time Machine* book using backpropagation through time for RNNs:

* The first row is the randomized truncation that partitions the text into segments of varying lengths.
* The second row is the regular truncation that breaks the text into subsequences of the same length. This is what we have been doing in RNN experiments.
* The third row is the full backpropagation through time that leads to a computationally infeasible expression.


Unfortunately, while appealing in theory, randomized truncation does not work much better than regular truncation, most likely due to a number of factors.
First, the effect of an observation after a number of backpropagation steps into the past is quite sufficient to capture dependencies in practice. 
Second, the increased variance counteracts the fact that the gradient is more accurate with more steps. 
Third, we actually *want* models that have only a short range of interactions. Hence, regularly truncated backpropagation through time has a slight regularizing effect that can be desirable.

## Backpropagation Through Time in Detail

After discussing the general principle,
let us discuss backpropagation through time in detail.
Different from the analysis in
:numref:`subsec_bptt_analysis`,
in the following
we will show
how to compute
the gradients of the objective function
with respect to all the decomposed model parameters.
To keep things simple, we consider 
an RNN without bias parameters,
whose 
activation function
in the hidden layer
uses the identity mapping ($\phi(x)=x$).
For time step $t$,
let the single example input and the label be
$\mathbf{x}_t \in \mathbb{R}^d$ and $y_t$, respectively. 
The hidden state $\mathbf{h}_t \in \mathbb{R}^h$ 
and the output $\mathbf{o}_t \in \mathbb{R}^q$
are computed as

$$\begin{aligned}\mathbf{h}_t &= \mathbf{W}_{hx} \mathbf{x}_t + \mathbf{W}_{hh} \mathbf{h}_{t-1},\\
\mathbf{o}_t &= \mathbf{W}_{qh} \mathbf{h}_{t},\end{aligned}$$

where $\mathbf{W}_{hx} \in \mathbb{R}^{h \times d}$, $\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$, and
$\mathbf{W}_{qh} \in \mathbb{R}^{q \times h}$
are the weight parameters.
Denote by $l(\mathbf{o}_t, y_t)$
the loss at time step $t$. 
Our objective function,
the loss over $T$ time steps
from the beginning of the sequence
is thus

$$L = \frac{1}{T} \sum_{t=1}^T l(\mathbf{o}_t, y_t).$$


In order to visualize the dependencies among
model variables and parameters during computation
of the RNN,
we can draw a computational graph for the model,
as shown in :numref:`fig_rnn_bptt`.
For example, the computation of the hidden states of time step 3, $\mathbf{h}_3$, depends on the model parameters $\mathbf{W}_{hx}$ and $\mathbf{W}_{hh}$,
the hidden state of the last time step $\mathbf{h}_2$,
and the input of the current time step $\mathbf{x}_3$.

![Computational graph showing dependencies for an RNN model with three time steps. Boxes represent variables (not shaded) or parameters (shaded) and circles represent operators.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/rnn-bptt.svg)
:label:`fig_rnn_bptt`

As just mentioned, the model parameters in :numref:`fig_rnn_bptt` are $\mathbf{W}_{hx}$, $\mathbf{W}_{hh}$, and $\mathbf{W}_{qh}$. 
Generally,
training this model
requires 
gradient computation with respect to these parameters
$\partial L/\partial \mathbf{W}_{hx}$, $\partial L/\partial \mathbf{W}_{hh}$, and $\partial L/\partial \mathbf{W}_{qh}$.
According to the dependencies in :numref:`fig_rnn_bptt`,
we can traverse 
in the opposite direction of the arrows
to calculate and store the gradients in turn.
To flexibly express the multiplication
of matrices, vectors, and scalars of different shapes
in the chain rule,
we continue to use 
the 
$\text{prod}$ operator as described in
:numref:`sec_backprop`.


First of all,
differentiating the objective function
with respect to the model output
at any time step $t$
is fairly straightforward:

$$\frac{\partial L}{\partial \mathbf{o}_t} =  \frac{\partial l (\mathbf{o}_t, y_t)}{T \cdot \partial \mathbf{o}_t} \in \mathbb{R}^q.$$
:eqlabel:`eq_bptt_partial_L_ot`

Now, we can calculate the gradient of the objective function
with respect to
the parameter $\mathbf{W}_{qh}$
in the output layer:
$\partial L/\partial \mathbf{W}_{qh} \in \mathbb{R}^{q \times h}$. Based on :numref:`fig_rnn_bptt`, 
the objective function
$L$ depends on $\mathbf{W}_{qh}$ via $\mathbf{o}_1, \ldots, \mathbf{o}_T$. Using the chain rule yields

$$
\frac{\partial L}{\partial \mathbf{W}_{qh}}
= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{o}_t}, \frac{\partial \mathbf{o}_t}{\partial \mathbf{W}_{qh}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{o}_t} \mathbf{h}_t^\top,
$$

where $\partial L/\partial \mathbf{o}_t$
is given by :eqref:`eq_bptt_partial_L_ot`.

Next, as shown in :numref:`fig_rnn_bptt`,
at the final time step $T$
the objective function
$L$ depends on the hidden state $\mathbf{h}_T$ only via $\mathbf{o}_T$.
Therefore, we can easily find
the gradient 
$\partial L/\partial \mathbf{h}_T \in \mathbb{R}^h$
using the chain rule:

$$\frac{\partial L}{\partial \mathbf{h}_T} = \text{prod}\left(\frac{\partial L}{\partial \mathbf{o}_T}, \frac{\partial \mathbf{o}_T}{\partial \mathbf{h}_T} \right) = \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_T}.$$
:eqlabel:`eq_bptt_partial_L_hT_final_step`

It gets trickier for any time step $t < T$,
where the objective function $L$ depends on $\mathbf{h}_t$ via $\mathbf{h}_{t+1}$ and $\mathbf{o}_t$.
According to the chain rule,
the gradient of the hidden state
$\partial L/\partial \mathbf{h}_t \in \mathbb{R}^h$
at any time step $t < T$ can be recurrently computed as:


$$\frac{\partial L}{\partial \mathbf{h}_t} = \text{prod}\left(\frac{\partial L}{\partial \mathbf{h}_{t+1}}, \frac{\partial \mathbf{h}_{t+1}}{\partial \mathbf{h}_t} \right) + \text{prod}\left(\frac{\partial L}{\partial \mathbf{o}_t}, \frac{\partial \mathbf{o}_t}{\partial \mathbf{h}_t} \right) = \mathbf{W}_{hh}^\top \frac{\partial L}{\partial \mathbf{h}_{t+1}} + \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_t}.$$
:eqlabel:`eq_bptt_partial_L_ht_recur`

For analysis,
expanding the recurrent computation
for any time step $1 \leq t \leq T$
gives

$$\frac{\partial L}{\partial \mathbf{h}_t}= \sum_{i=t}^T {\left(\mathbf{W}_{hh}^\top\right)}^{T-i} \mathbf{W}_{qh}^\top \frac{\partial L}{\partial \mathbf{o}_{T+t-i}}.$$
:eqlabel:`eq_bptt_partial_L_ht`

We can see from :eqref:`eq_bptt_partial_L_ht` that
this simple linear example already
exhibits some key problems of long sequence models: it involves potentially very large powers of $\mathbf{W}_{hh}^\top$.
In it, eigenvalues smaller than 1 vanish
and eigenvalues larger than 1 diverge.
This is numerically unstable,
which manifests itself in the form of vanishing 
and exploding gradients.
One way to address this is to truncate the time steps
at a computationally convenient size as discussed in :numref:`subsec_bptt_analysis`. 
In practice, this truncation is effected by detaching the gradient after a given number of time steps.
Later on 
we will see how more sophisticated sequence models such as long short-term memory can alleviate this further. 

Finally,
:numref:`fig_rnn_bptt` shows that
the objective function
$L$ depends on model parameters
$\mathbf{W}_{hx}$ and $\mathbf{W}_{hh}$
in the hidden layer
via hidden states
$\mathbf{h}_1, \ldots, \mathbf{h}_T$.
To compute gradients
with respect to such parameters
$\partial L / \partial \mathbf{W}_{hx} \in \mathbb{R}^{h \times d}$ and $\partial L / \partial \mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$,
we apply the chain rule that gives

$$
\begin{aligned}
\frac{\partial L}{\partial \mathbf{W}_{hx}}
&= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{h}_t}, \frac{\partial \mathbf{h}_t}{\partial \mathbf{W}_{hx}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h}_t} \mathbf{x}_t^\top,\\
\frac{\partial L}{\partial \mathbf{W}_{hh}}
&= \sum_{t=1}^T \text{prod}\left(\frac{\partial L}{\partial \mathbf{h}_t}, \frac{\partial \mathbf{h}_t}{\partial \mathbf{W}_{hh}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h}_t} \mathbf{h}_{t-1}^\top,
\end{aligned}
$$

where
$\partial L/\partial \mathbf{h}_t$
that is recurrently computed by
:eqref:`eq_bptt_partial_L_hT_final_step`
and
:eqref:`eq_bptt_partial_L_ht_recur`
is the key quantity
that affects the numerical stability.



Since backpropagation through time
is the application of backpropagation in RNNs,
as we have explained in :numref:`sec_backprop`,
training RNNs
alternates forward propagation with
backpropagation through time.
Besides,
backpropagation through time
computes and stores the above gradients
in turn.
Specifically,
stored intermediate values
are reused
to avoid duplicate calculations,
such as storing 
$\partial L/\partial \mathbf{h}_t$
to be used in computation of both $\partial L / \partial \mathbf{W}_{hx}$ and $\partial L / \partial \mathbf{W}_{hh}$.


## Summary

* Backpropagation through time is merely an application of backpropagation to sequence models with a hidden state.
* Truncation is needed for computational convenience and numerical stability, such as regular truncation and randomized truncation.
* High powers of matrices can lead to divergent or vanishing eigenvalues. This manifests itself in the form of exploding or vanishing gradients.
* For efficient computation, intermediate values are cached during backpropagation through time.



## Exercises

1. Assume that we have a symmetric matrix $\mathbf{M} \in \mathbb{R}^{n \times n}$ with eigenvalues $\lambda_i$ whose corresponding eigenvectors are $\mathbf{v}_i$ ($i = 1, \ldots, n$). Without loss of generality, assume that they are ordered in the order $|\lambda_i| \geq |\lambda_{i+1}|$. 
   1. Show that $\mathbf{M}^k$ has eigenvalues $\lambda_i^k$.
   1. Prove that for a random vector $\mathbf{x} \in \mathbb{R}^n$, with high probability $\mathbf{M}^k \mathbf{x}$ will be very much aligned with the eigenvector $\mathbf{v}_1$ 
of $\mathbf{M}$. Formalize this statement.
   1. What does the above result mean for gradients in RNNs?
1. Besides gradient clipping, can you think of any other methods to cope with gradient explosion in recurrent neural networks?